- Notebook demonstrate how to implement GAN from scratch using Keras, tensorflow. 
- Using GAN to generate digit image (learning from MNIST dataset).

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *

# Implement GAN

In [2]:
class DCGAN:

    @staticmethod
    def build_generator(dim, depth, channels=1, inputDim=100, outputDim=512):

        model = Sequential()
        inputShape = (dim, dim, depth)
        chanDim = -1

        model.add(Dense(input_dim=inputDim, units=outputDim))
        model.add(Activation("relu"))
        model.add(BatchNormalization())

        model.add(Dense(dim * dim * depth))
        model.add(Activation("relu"))
        model.add(BatchNormalization())

        model.add(Reshape(inputShape))

        model.add(Conv2DTranspose(32, (5, 5), strides=(2, 2), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))

        model.add(Conv2DTranspose(channels, (5, 5), strides=(2, 2), padding="same"))
        model.add(Activation("tanh"))
        
        return model

    @staticmethod
    def build_discriminator(width, height, depth, alpha=0.2):

        model = Sequential()

        inputShape = (height, width, depth)

        model.add(Conv2D(32, (5, 5), padding="same", strides=(2, 2), input_shape=inputShape))
        model.add(LeakyReLU(alpha=alpha))

        model.add(Conv2D(64, (5, 5), padding="same", strides=(2, 2)))
        model.add(LeakyReLU(alpha=alpha))

        model.add(Flatten())
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=alpha))

        model.add(Dense(1))
        model.add(Activation("sigmoid"))

        return model

# Training GAN

In [3]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import fashion_mnist, mnist
from sklearn.utils import shuffle
from imutils import build_montages
import numpy as np
import argparse
import cv2
import os

In [4]:
OUTPUT_PATH = "output"
NUM_EPOCHS = 20
BATCH_SIZE = 128
INIT_LR = 2e-4

! mkdir "output"

mkdir: cannot create directory ‘output’: File exists


In [5]:
# Load MNIST dataset

((trainX, _), (testX, _)) = mnist.load_data()
trainImages = np.concatenate([trainX, testX])

trainImages = np.expand_dims(trainImages, axis=-1)

trainImages = (trainImages.astype("float") - 127.5) / 127.5

In [6]:
# Build generator model
gen = DCGAN.build_generator(7, 64, channels=1)

# Build discriminator model
disc = DCGAN.build_discriminator(28, 28, 1)
discOpt = Adam(learning_rate=INIT_LR, beta_1=0.5, decay=INIT_LR / NUM_EPOCHS)
disc.compile(loss="binary_crossentropy", optimizer=discOpt)

In [7]:
# Build GAN
disc.trainable = False  # FREEZE PARAMETERE
ganInput = Input(shape=(100,))
ganOutput = disc(gen(ganInput))
gan = Model(ganInput, ganOutput)

# compile the GAN
ganOpt = Adam(learning_rate=INIT_LR, beta_1=0.5, decay=INIT_LR / NUM_EPOCHS)
gan.compile(loss="binary_crossentropy", optimizer=discOpt)

In [8]:
# Training process
benchmarkNoise = np.random.uniform(-1, 1, size=(64, 100))

# loop over the epochs
for epoch in range(0, NUM_EPOCHS):
    print("[INFO] starting epoch {} of {}...".format(epoch + 1, NUM_EPOCHS))
    batchesPerEpoch = int(trainImages.shape[0] / BATCH_SIZE)

    for i in range(0, batchesPerEpoch):
        
        p = None
        imageBatch = trainImages[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
        noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))

        # Generator generate image
        genImages = gen.predict(noise, verbose=0)

        # Concat "real" images and "'fake" images
        X = np.concatenate((imageBatch, genImages))
        y = ([1] * BATCH_SIZE) + ([0] * BATCH_SIZE)
        y = np.reshape(y, (-1,))
        (X, y) = shuffle(X, y)

        # train the discriminator on the data
        discLoss = disc.train_on_batch(X, y)

        # Inverted label
        noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
        fakeLabels = [1] * BATCH_SIZE
        fakeLabels = np.reshape(fakeLabels, (-1,))
        ganLoss = gan.train_on_batch(noise, fakeLabels)

        # End of Epoch
        if i == batchesPerEpoch - 1:
			
            p = [OUTPUT_PATH, "epoch_{}_output.png".format(str(epoch + 1).zfill(4))]

            print("[INFO] Step {}_{}: discriminator_loss={:.6f}, ""adversarial_loss={:.6f}".format(epoch + 1, i, discLoss, ganLoss))

            # make predictions on the benchmark noise, scale it back to [0, 255], and generate the montage
            images = gen.predict(benchmarkNoise)
            images = ((images * 127.5) + 127.5).astype("uint8")
            images = np.repeat(images, 3, axis=-1)
            vis = build_montages(images, (28, 28), (8, 8))[0]

            # write the visualization to disk
            p = os.path.sep.join(p)
            print("path: ", p)
            cv2.imwrite(p, vis)

[INFO] starting epoch 1 of 20...
[INFO] Step 1_545: discriminator_loss=0.297950, adversarial_loss=1.609284
path:  output/epoch_0001_output.png
[INFO] starting epoch 2 of 20...
[INFO] Step 2_545: discriminator_loss=0.453995, adversarial_loss=1.509171
path:  output/epoch_0002_output.png
[INFO] starting epoch 3 of 20...
[INFO] Step 3_545: discriminator_loss=0.492564, adversarial_loss=1.685188
path:  output/epoch_0003_output.png
[INFO] starting epoch 4 of 20...
[INFO] Step 4_545: discriminator_loss=0.504698, adversarial_loss=1.195726
path:  output/epoch_0004_output.png
[INFO] starting epoch 5 of 20...
[INFO] Step 5_545: discriminator_loss=0.500017, adversarial_loss=1.659174
path:  output/epoch_0005_output.png
[INFO] starting epoch 6 of 20...
[INFO] Step 6_545: discriminator_loss=0.440981, adversarial_loss=1.431539
path:  output/epoch_0006_output.png
[INFO] starting epoch 7 of 20...
[INFO] Step 7_545: discriminator_loss=0.472230, adversarial_loss=1.475965
path:  output/epoch_0007_output.png